# MyScale

This notebook shows how to use functionality related to the MyScale vector database.

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import MyScale
from langchain.document_loaders import TextLoader

## Setting up envrionments

There are two ways to set up parameters for myscale index.

1. Environment Variables

    Before you run the app, please set the environment variable with `export`:

    `export MYSCALE_URL='<your-endpoints-url>' MYSCALE_PORT=<your-endpoints-port> MYSCALE_USERNAME=<your-username> MYSCALE_PASSWORD=<your-password> ...`

    Every attributes under `MyScaleSettings` can be set with prefix `MYSCALE_` and is case insensitive.

2. Create `MyScaleSettings` object with parameters


    ```python
    from langchain.vectorstores import MyScale, MyScaleSettings
    config = MyScaleSetting(host="localhost", port=8123, ...)
    index = MyScale(embedding_function, config)
    index.add_documents(...)
    ```

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [ ]:
for d in docs:
    d.metadata = {'some': 'metadata'}
docsearch = MyScale.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = docsearch.similarity_search(query)

In [ ]:
print(docs[0].page_content)

## Show data schema

In [ ]:
print(str(docsearch))

## Filtering

You can have direct access to myscale SQL where statement. You can write `WHERE` clause following standard SQL.

**NOTE**: Please be aware of SQL injection, this interface must not be directly called by end-user.

If you custimized your `column_map` under your setting, you search with filter like this:

In [ ]:
from langchain.vectorstores import MyScale, MyScaleSettings
from langchain.document_loaders import TextLoader

loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

for d in docs:
    d.metadata = {'some': 'metadata'}

docsearch = MyScale.from_documents(docs, embeddings)

In [7]:
meta = docsearch.metadata_column
docs = docsearch.similarity_search_with_relevance_scores('What did the president say about Ketanji Brown Jackson?', 
    where_str=f"{meta}.some='metadata'")

## On deleting your data

In [ ]:
docsearch.drop()